In [66]:
from diplomacy_research.utils.tensorflow import tf
from diplomacy_research.models.datasets.base_builder import BaseBuilder
from diplomacy_research.models.policy.order_based.dataset import no_press_all
import pickle
import numpy as np

from diplomacy_research.models.state_space import get_adjacency_matrix
from diplomacy_research.models.layers.graph_convolution import film_gcn_res_block, preprocess_adjacency

In [67]:
tf.enable_eager_execution()

In [68]:
from diplomacy_research.models.layers.dropout import SeededDropoutWrapper
from diplomacy_research.models.layers.seeded_random import seeded_random

In [69]:
lstm_cell = tf.contrib.rnn.LSTMBlockCell(10)

In [70]:
inputs = tf.ones(shape=(5, 14))

In [71]:
output, cell_state = lstm_cell(inputs, lstm_cell.zero_state(5, dtype=float))

In [78]:
output += 1

In [79]:
cell_state[1]

<tf.Tensor: id=395, shape=(5, 10), dtype=float32, numpy=
array([[ 0.22209373, -0.03357967, -0.02534223,  0.04242827, -0.13270867,
        -0.07274736,  0.01204854, -0.12289876,  0.0889146 , -0.1324391 ],
       [ 0.22209373, -0.03357967, -0.02534223,  0.04242827, -0.13270867,
        -0.07274736,  0.01204854, -0.12289876,  0.0889146 , -0.1324391 ],
       [ 0.22209373, -0.03357967, -0.02534223,  0.04242827, -0.13270867,
        -0.07274736,  0.01204854, -0.12289876,  0.0889146 , -0.1324391 ],
       [ 0.22209373, -0.03357967, -0.02534223,  0.04242827, -0.13270867,
        -0.07274736,  0.01204854, -0.12289876,  0.0889146 , -0.1324391 ],
       [ 0.22209373, -0.03357967, -0.02534223,  0.04242827, -0.13270867,
        -0.07274736,  0.01204854, -0.12289876,  0.0889146 , -0.13243912]],
      dtype=float32)>

In [80]:
output

<tf.Tensor: id=405, shape=(5, 10), dtype=float32, numpy=
array([[1.2220937 , 0.96642035, 0.9746578 , 1.0424283 , 0.86729133,
        0.92725265, 1.0120485 , 0.87710124, 1.0889146 , 0.86756086],
       [1.2220937 , 0.96642035, 0.9746578 , 1.0424283 , 0.86729133,
        0.92725265, 1.0120485 , 0.87710124, 1.0889146 , 0.86756086],
       [1.2220937 , 0.96642035, 0.9746578 , 1.0424283 , 0.86729133,
        0.92725265, 1.0120485 , 0.87710124, 1.0889146 , 0.86756086],
       [1.2220937 , 0.96642035, 0.9746578 , 1.0424283 , 0.86729133,
        0.92725265, 1.0120485 , 0.87710124, 1.0889146 , 0.86756086],
       [1.2220937 , 0.96642035, 0.9746578 , 1.0424283 , 0.86729133,
        0.92725265, 1.0120485 , 0.87710124, 1.0889146 , 0.86756086]],
      dtype=float32)>

In [6]:
from diplomacy_research.utils.tensorflow import rnn_cell_impl
enum_map_up_to = rnn_cell_impl._enumerated_map_structure_up_to

In [128]:
values = np.array([[1,2], [1,2]])

In [129]:
# values = tf.ones(shape=(128, 200)).numpy()

In [130]:
filtered_structure = values

In [131]:
filtered_structure

array([[1, 2],
       [1, 2]])

In [132]:
def func(c, a, b):
    return f"bruh {c} -- {a} -- {b}"

In [133]:
enum_map_up_to(values, func, *[values, filtered_structure])

'bruh 0 -- [[1 2]\n [1 2]] -- [[1 2]\n [1 2]]'

In [5]:
player_seeds = tf.constant([0, 0, 0, 1, 0, 2, 2], dtype = tf.int32)
dropout_rates = tf.zeros(shape=(7))

In [11]:
player_seeds + 1

<tf.Tensor: id=20, shape=(7,), dtype=int32, numpy=array([1, 1, 1, 2, 1, 3, 3], dtype=int32)>

Unique(y=<tf.Tensor: id=15, shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>, idx=<tf.Tensor: id=16, shape=(7,), dtype=int32, numpy=array([0, 0, 0, 1, 0, 2, 2], dtype=int32)>)

In [ ]:
player_seeds = tf.random.uniform(
    [128],
    minval=0,
    maxval=2,
    dtype=tf.dtypes.int32,
    seed=42,
)

In [29]:
player_seeds

<tf.Tensor: id=64, shape=(128,), dtype=int32, numpy=
array([1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1], dtype=int32)>

In [23]:
player_seeds[0].insert(2)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'insert'

In [6]:
decoder_cell = SeededDropoutWrapper(cell=lstm_cell,
                                    seeds=player_seeds,
                                    input_keep_probs=1. - dropout_rates,
                                    output_keep_probs=1. - dropout_rates,
                                    variational_recurrent=True,
                                    input_size=7,
                                    dtype=tf.float32)

In [7]:
print(decoder_cell._recurrent_input_noise)
print(tf.shape(decoder_cell._recurrent_input_noise))

tf.Tensor(
[[0.5137526  0.23870096 0.46468195 0.17104298 0.7501834  0.746515
  0.81621486]
 [0.5407327  0.7260785  0.20450804 0.11434726 0.827402   0.27936253
  0.6921117 ]
 [0.2827031  0.6286411  0.05581885 0.93944186 0.15060474 0.13850984
  0.36831293]
 [0.76637125 0.43894675 0.6600114  0.45978346 0.26411387 0.98183614
  0.34511364]
 [0.7580364  0.5973092  0.6511253  0.62861854 0.0562475  0.9312974
  0.30534926]
 [0.7663712  0.43894693 0.6600084  0.45984098 0.26302156 0.00259055
  0.9507796 ]
 [0.7663712  0.43894693 0.6600084  0.45984098 0.26302156 0.00259055
  0.9507796 ]], shape=(7, 7), dtype=float32)
tf.Tensor([7 7], shape=(2,), dtype=int32)


In [8]:
print(decoder_cell._recurrent_output_noise)
print(tf.shape(decoder_cell._recurrent_output_noise))

tf.Tensor(
[[0.5884315  0.8198015  0.42377096 ... 0.39512742 0.49257907 0.6409975 ]
 [0.4639374  0.18518937 0.48140204 ... 0.73964626 0.9467216  0.01228966]
 [0.6855252  0.97502196 0.4745832  ... 0.6554763  0.5459505  0.6269404 ]
 ...
 [0.31320372 0.0491292  0.06654508 ... 0.47173628 0.03701064 0.29679784]
 [0.26088855 0.04311738 0.18076973 ... 0.6998935  0.7020235  0.6615534 ]
 [0.26088855 0.04311738 0.18076973 ... 0.6998935  0.7020235  0.6615534 ]], shape=(7, 200), dtype=float32)
tf.Tensor([  7 200], shape=(2,), dtype=int32)


In [15]:
from diplomacy_research.utils.tensorflow import _transpose_batch_time
from diplomacy_research.utils.tensorflow import _unstack_ta

In [44]:
from diplomacy_research.utils.tensorflow import array_ops

In [45]:
bruh = tf.constant([
    [[ 1,  2,  3], [ 1,  2,  3]],
    [[10, 11, 12], [ 1,  2,  3]],
    [[16, 17, 18], [ 1,  2,  3]]
])

In [47]:
bruh

<tf.Tensor: id=186, shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 1,  2,  3]],

       [[10, 11, 12],
        [ 1,  2,  3]],

       [[16, 17, 18],
        [ 1,  2,  3]]], dtype=int32)>

In [46]:
array_ops.identity(bruh)

<tf.Tensor: id=186, shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 1,  2,  3]],

       [[10, 11, 12],
        [ 1,  2,  3]],

       [[16, 17, 18],
        [ 1,  2,  3]]], dtype=int32)>

In [17]:
print(bruh)

tf.Tensor(
[[[ 1  2  3]
  [ 1  2  3]]

 [[10 11 12]
  [ 1  2  3]]

 [[16 17 18]
  [ 1  2  3]]], shape=(3, 2, 3), dtype=int32)


In [18]:
x = _transpose_batch_time(bruh)

In [19]:
print(x)

tf.Tensor(
[[[ 1  2  3]
  [10 11 12]
  [16 17 18]]

 [[ 1  2  3]
  [ 1  2  3]
  [ 1  2  3]]], shape=(2, 3, 3), dtype=int32)


In [20]:
g = _unstack_ta(x)

In [21]:
g.size()

<tf.Tensor: id=131, shape=(), dtype=int32, numpy=2>

In [23]:
g.read(0)

<tf.Tensor: id=129, shape=(3, 3), dtype=int32, numpy=
array([[ 1,  2,  3],
       [10, 11, 12],
       [16, 17, 18]], dtype=int32)>

In [108]:
cell = tf.contrib.rnn.LSTMBlockCell(200)

In [109]:
print(cell.zero_state(128, float)[0])

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(128, 200), dtype=float32)


In [118]:
from diplomacy_research.utils.tensorflow import math_ops

In [121]:
a = tf.random.uniform(shape=[128, 1, 81])
b = tf.random.uniform(shape=[128, 81, 120])

In [122]:
math_ops.matmul(a, b)

<tf.Tensor: id=466, shape=(128, 1, 120), dtype=float32, numpy=
array([[[19.383259, 21.408987, 18.56496 , ..., 18.423637, 21.74904 ,
         18.488977]],

       [[17.560833, 19.754374, 21.891068, ..., 18.670254, 20.508831,
         18.745518]],

       [[21.16572 , 20.09104 , 23.58803 , ..., 20.126144, 23.200521,
         22.395763]],

       ...,

       [[20.50752 , 20.468699, 22.060888, ..., 23.730959, 21.717173,
         21.441353]],

       [[21.957153, 21.998652, 19.98428 , ..., 17.20297 , 23.759792,
         23.022839]],

       [[16.641558, 15.036928, 17.367977, ..., 17.630404, 17.319685,
         17.948994]]], dtype=float32)>

In [3]:
dataset_builder = no_press_all.DatasetBuilder()

In [4]:
training_dataset = tf.data.TFRecordDataset("20190501-proto-policy-order_based-no_press_all.train.pbz",
                                                        compression_type='GZIP')

2022-04-24 10:02:01.765787: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-24 10:02:01.770645: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-04-24 10:02:01.771708: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x556c93554040 executing computations on platform Host. Devices:
2022-04-24 10:02:01.771735: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


In [5]:
# training_dataset = training_dataset.shuffle(100)

In [6]:
training_dataset = training_dataset.map(dataset_builder.parse_function, num_parallel_calls=2)

In [7]:
dataset_builder.output_shapes

{'request_id': [],
 'player_seed': [],
 'board_state': [81, 35],
 'board_alignments': [None],
 'prev_orders_state': [1, 81, 40],
 'decoder_inputs': [None],
 'decoder_lengths': [],
 'candidates': [None],
 'noise': [],
 'temperature': [],
 'dropout_rate': [],
 'current_power': [],
 'current_season': [],
 'draw_target': [],
 'value_target': []}

In [8]:
# training_dataset = training_dataset.prefetch(100)

In [9]:
proto_fields = dataset_builder.parse_sparse_fields(dataset_builder.proto_fields)

In [10]:
training_dataset = training_dataset.padded_batch(2, dataset_builder.padded_shapes)

In [11]:
i = 0
for element in training_dataset:
    # print(element)
    # print(element.keys())
    # print(element["request_id"])
    # print(element["player_seed"])
    # print(element["board_state"])
    # print(element["board_alignments"])
    # print(element["prev_orders_state"])
    # print(element["decoder_inputs"])
    print(element["decoder_lengths"])
    # print(element["candidates"])
    # print(element["noise"])
    # print(element["temperature"])
    # print(element["dropout_rate"])
    # print(element["current_power"])
    # print(element["current_season"])
    # print(element["draw_target"])
    # print(element["value_target"])
    # print("****")
    
    # print(element["candidates"])
    
    if (i == 4):
        break
    i+=1

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


tf.Tensor([3 3], shape=(2,), dtype=int32)
tf.Tensor([3 3], shape=(2,), dtype=int32)
tf.Tensor([3 1], shape=(2,), dtype=int32)
tf.Tensor([6 4], shape=(2,), dtype=int32)
tf.Tensor([6 4], shape=(2,), dtype=int32)


#### Model

In [1]:
from diplomacy_research.utils.model import load_dynamically_from_model_id, load_dataset_builder
from diplomacy_research.models.training.supervised import SupervisedTrainer
from diplomacy_research.models.policy.order_based import PolicyAdapter
from diplomacy_research.utils.model import parse_args_into_flags,load_dataset_builder
from diplomacy_research.settings import ROOT_DIR
from diplomacy_research.utils.cluster import start_distributed_training
from diplomacy_research.utils.cluster_config.supervised import get_cluster_config
import argparse
import os
from diplomacy_research.utils.tensorflow import tf

2022-03-30 08:02:02,844 - diplomacy_research.utils.tensorflow - INFO - TensorFlow has been successfully loaded.


In [2]:
tf.enable_eager_execution()

In [3]:
PolicyModel, load_args = load_dynamically_from_model_id(
    ['PolicyModel', 'load_args'],
    base_dir = "/home/jupyter/rl_ppaquette/diplomacy_research/models/policy/order_based",
    arg_name='model_id', model_id = 12) 


# Detecting ValueModel, and load_args
ValueModel, load_value_args = load_dynamically_from_model_id(['ValueModel', 'load_args'],                         
                                                             arg_name='value_model_id',
                                                             model_id = 12,
                                                             base_dir=os.path.join(ROOT_DIR, 'models', 'value'),
                                                             on_error='ignore')

# Detecting DrawModel, and load_args
DrawModel, load_draw_args = load_dynamically_from_model_id(['DrawModel', 'load_args'],                           
                                                           arg_name='draw_model_id',
                                                           model_id= 12,
                                                           base_dir=os.path.join(ROOT_DIR, 'models', 'draw'),
                                                           on_error='ignore')

2022-03-30 08:02:04,745 - diplomacy_research.utils.model - INFO - Loading class: diplomacy_research.models.policy.order_based.v012_film_diff_w_board_align_prev_ord.PolicyModel
2022-03-30 08:02:04,747 - diplomacy_research.utils.model - INFO - Loading class: diplomacy_research.models.policy.order_based.v012_film_diff_w_board_align_prev_ord.load_args


In [4]:
# load_args()

In [5]:
FLAGS = argparse.Namespace(
    allow_gpu_growth=False,
    attn_size=120,
    avg_gradients=False,
    batch_size=128,
    beam_groups=5,
    beam_width=10,
    config='config.yaml',
    config_section='',
    dataset='no_press_all',
    debug=False,
    debug_batch=False,
    debug_nb_cpu_workers=0,
    dropout_rate=0.5,
    early_stopping_stop_after=5,
    gcn_size=120, gpu_id=-1,
    grad_aggregation='ADD_N',
    gradient_checkpoint='',
    grpc_port=2200,
    learning_rate=0.001,
    lr_decay_factor=0.93,
    lstm_size=200, 
    max_gradient_norm=5.0,
    min_nb_param_servers=0,
    model_id=12,
    nb_graph_conv=16,
    nb_param_servers=3, 
    order_emb_size=80,
    perc_epoch_for_training=1.0,
    policy_coeff=1.0, 
    power_emb_size=60, 
    profile='',
    save_dir='/home/jupyter/rl_ppaquette/test_setup/2022-03-30_saved_supervised',
    season_emb_size=20,
    swap_memory=False,
    sync_gradients=True,
    training_mode='supervised',
    use_partitioner=False,
    use_v_dropout=True,
    use_verbs=False,
    use_xla=True)

In [6]:
dataset = load_dataset_builder(FLAGS.dataset, base_dir="/home/jupyter/rl_ppaquette/diplomacy_research/models/policy/order_based")

2022-03-30 08:02:06,026 - diplomacy_research.utils.model - INFO - Loading dataset "no_press_all" using diplomacy_research.models.policy.order_based.dataset.no_press_all.DatasetBuilder


In [7]:
def get_trainer(cluster_config, _):
    SupervisedTrainer(policy_constructor=PolicyModel,
    value_constructor=None,
                          draw_constructor=None,
                          dataset_builder = dataset(),
                          adapter_constructor=PolicyAdapter,
                          cluster_config=cluster_config,
                          flags=FLAGS).start()

In [8]:
start_distributed_training(callable_fn=get_trainer,
                           flags=FLAGS,
                           get_cluster_config_fn=get_cluster_config,
                           with_process_pool=False)

2022-03-30 08:02:06,928 - diplomacy_research.utils.cluster - INFO - Received command line arguments: /opt/conda/envs/diplomacy/lib/python3.7/site-packages/ipykernel_launcher.py -f /home/jupyter/.local/share/jupyter/runtime/kernel-ba9292aa-e97d-4e5d-879d-751b4437c6d9.json
2022-03-30 08:02:06,929 - diplomacy_research.utils.cluster - INFO - CUDA_VISIBLE_DEVICE: <Not set>
2022-03-30 08:02:06,930 - diplomacy_research.utils.model - INFO - ----------------------------------------
2022-03-30 08:02:06,931 - diplomacy_research.utils.model - INFO - Commit Hash: 6f6b4bd --- *** GIT WORKING TREE IS DIRTY ***
2022-03-30 08:02:06,932 - diplomacy_research.utils.model - INFO - ----------------------------------------
2022-03-30 08:02:06,932 - diplomacy_research.utils.model - INFO - Flag allow_gpu_growth: False
2022-03-30 08:02:06,933 - diplomacy_research.utils.model - INFO - Flag attn_size: 120
2022-03-30 08:02:06,933 - diplomacy_research.utils.model - INFO - Flag avg_gradients: False
2022-03-30 08:02:

2022-03-30 08:02:06.962554: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-03-30 08:02:06.967960: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2022-03-30 08:02:06.969276: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55e7d586ab60 executing computations on platform Host. Devices:
2022-03-30 08:02:06.969310: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>


FailedPreconditionError: GetNext() failed because the iterator has not been initialized. Ensure that you have run the initializer operation for this iterator before getting the next element. [Op:IteratorGetNext]